In [13]:
import csv
import io
import os
from PIL import Image
import random
import requests
import selenium
from selenium import webdriver
import time

In [14]:
def collect_thumbnails(wd, n_skip = 0):
    if n_skip != 0 : is_load_button(wd)
    scroll_to_end(wd)
    thumbnails = wd.find_elements_by_css_selector("img.Q4LuWd")
    print('There are currently {} thumbnails selected'.format(len(thumbnails)))
    return thumbnails[n_skip:]

def is_load_button(wd):
    time.sleep(7)
    load_more_button = wd.find_element_by_css_selector(".mye4qd")
    if load_more_button:
        print('Load button found. Loading new images now.')
        time.sleep(3)
        wd.execute_script("document.querySelector('.mye4qd').click();")
        

In [15]:
def collect_thumbnail_urls(wd, thumbnails):
    urls = set()
    for tb in thumbnails:
        try:
            tb.click()
            time.sleep(3)
        except:
            print('Unable to select element: stale reference or insufficient load time')
            continue
        full_imgs = wd.find_elements_by_css_selector('img.n3VNCb')
        for img in full_imgs:
            try:
                url = img.get_attribute('src')
                if url and 'http' in url: 
                    urls.add(url)
            except:
                print('Unable to capture URL from full_size image. Trying the next image')
    return urls

#helper method to identify when a google search result is maxed out
def count_total_thumbnails(wd):
    thumbnails = wd.find_elements_by_css_selector("img.Q4LuWd")
    return len(thumbnails)

In [62]:
def collect_image_urls(query, n_urls, rest_time, wd):
    
    urls = set()

    #Google Query Construction
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # Load the page
    wd.get(search_url.format(q=query))

    scroll_placeholder = 0
    print('urls length = {}'.format(len(urls)))

    while len(urls) < n_urls and scroll_placeholder <  count_total_thumbnails(wd):
        print('n_urls equals ' + str(n_urls))
        print('the total number of thumbnails is {}. The scroll placeholder variable is {}'.format(count_total_thumbnails(wd), scroll_placeholder))
        print('urls length = {}'.format(len(urls)))
        print("Scanned {} thumbnails".format(scroll_placeholder))
        thumbnails = collect_thumbnails(wd, scroll_placeholder)
        batch_urls = collect_thumbnail_urls(wd, thumbnails)
        scroll_placeholder += len(thumbnails)
        urls = urls.union(batch_urls)
    print ('Completed scraping images for {} . Successfully scraped {} image urls!'.format(query, len(urls)))
    return urls

In [66]:
# fix here
def save_image(url, cd, key, query):

    try:
        response = requests.get(url).content
        
    except:
        print('Unable to download image from {}...\n Moving on to next url'.format(url[:20]))
    
    
    random_ext = random.randint(1000000, 9999999)
    

    try:
        image_path = cd + '/' 'images' + '/' + key + '/' + query + '/' + query + '_' + str(random_ext) + '.jpg'
        image_file = io.BytesIO(response)
        image = Image.open(image_file).convert('RGB')
        
        try:
            with open (image_path, 'wb') as img:
                image.save(img, "JPEG", quality = 85)
                print('Saved image to {}'.format(image_path))
                
        except Exception as e:
            print('Issue with saving image content. Error: {}'.format(e))
            print(image_path)
            
        
    except Exception as e:
        print('Unable to complete save: {} \n Moving onto next image...'.format(e))

#test Save Image
url = 'https://www.thespruce.com/thmb/iLMS8yDu2YpdkxM2YW9uI6rOS4U=/2000x2000/smart/filters:no_upscale()/purple-lilac-big-56a582865f9b58b7d0dd3a97.jpg'
cd = '/Users/ems/Desktop/full_scraping_test'
save_image(url, cd, 'flowers', 'lilac')
# /Users/ems/Desktop/complete/images/flowers/lilac

Saved image to /Users/ems/Desktop/full_scraping_test/images/flowers/lilac/lilac_2465126.jpg


In [51]:
#Helper Methods
def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    
def create_directories(cd, key, values):
    root_images = cd + '/' + 'images'
    root_images_with_key = root_images + '/' + key
    root_csvs = cd + '/' + 'csvs'
    
    for path in [cd, root_images, root_images_with_key, root_csvs]:
        if not os.path.exists(path):
            print('No path at {}. \n Creating new directory!'.format(cd))
            os.mkdir(path)
    for v in values:
        print(v)
        query_image_file= root_images_with_key + '/' + v
        if not os.path.exists(query_image_file):
            os.mkdir(query_image_file)

test_cd = '/Users/ems/Desktop/full_scraping_test'
query_dict = {"flowers": ['rose', 'lilac']}
create_directories(test_cd, 'key', ['query1', 'query2'])
# cd/images/flower/petunia10293846.png
# cd/csv/flower_petunia.csv


query1
query2


In [60]:
def save_urls_to_csv(arr, key, query, cd):
        
    query_as_csv = key.replace(' ', '_') + '_' + query.replace(' ', '_') + '.csv'
    file_path = (cd + '/' + 'csvs' + '/' + query_as_csv)
    
    op = 'a' if os.path.exists(file_path) else 'w'
        
    with open(file_path, op, newline = '') as output_csv:
        output_writer = csv.writer(output_csv)
        for item in arr:
            output_writer.writerow([item])
# save_urls_to_csv(['aaa', 'bbbb'], 'key', 'nubbles', test_cd)


To use this script:<br><br>
    1) Enter your google search term below:<br><br>
    2) Enter a path for storing images<br><br>
        -This script will create a directory if one does not exist.<br>
        -This directory will be named for the search term<br><br>
    3) The script works as follows:<br>
        -Scrolls through google search results and saves n unique URLs to a csv<br>
        -Revisits CSV and downloads image from each URL<br>
    <br><br>
    If you have multiple search terms, input them as a list

In [67]:
urls = set()
pause_time = 1
DRIVER_PATH = '/usr/local/bin/chromedriver'
cd = '/Users/ems/Desktop/full_scraping_test'
query_dict = {"guns": ['glock', 'AR-15']}
n_urls =30


def execute_program(query_dict, cd, n_urls):

    try:
        os.chdir(cd)
    except Exception as e:
        print(e)
        print('Current working directories: ' + os.getcwd())
        
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    

    try:
        for query in query_dict.items():
            urls = set()
            create_directories(cd, query[0], query[1])
            print('directiories created!')

            for v in query[1]:
                new_url_batch = collect_image_urls(v, n_urls, 5, wd)
                urls = urls.union(new_url_batch)
                save_urls_to_csv(urls, query[0], v, cd)
                
                for url in new_url_batch:
                    save_image(url, cd, query[0], v)
        
            
            
    except Exception as e:
        print(e)
        wd.quit()
    return 

execute_program(query_dict, cd, n_urls)
        


#Google implemented limits on all google image search results. Thus, multiple searches
# may need to be conducted with synonyms (ex. canine, dog). Queries can be passed as 
# key-value pairs (with the key acting as the directory name) or as a list. This should 
# allow for more efficient searching. 
# 


glock
AR-15
directiories created!
urls length = 0
n_urls equals 30
the total number of thumbnails is 48. The scroll placeholder variable is 0
urls length = 0
Scanned 0 thumbnails
There are currently 100 thumbnails selected
Unable to select element: stale reference or insufficient load time
Completed scraping images for glock . Successfully scraped 170 image urls!
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_7813961.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_5783398.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_9160081.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_4389055.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_5888760.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_3130964.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_2875031.jpg
Saved image

Traceback (most recent call last):
  File "/Users/ems/.pyenv/versions/3.8.5/lib/python3.8/site-packages/urllib3/connectionpool.py", line 446, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/Users/ems/.pyenv/versions/3.8.5/lib/python3.8/site-packages/urllib3/util/response.py", line 71, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'X-Frame-Options : SAMEORIGIN\r\nX-Content-Type-Options : nosniff\r\nX-Xss-Protection : 1; mode=block\r\nDate: Fri, 06 Nov 2020 13:39:35 GMT\r\nContent-Length: 481869\r\n\r\n'


Unable to download image from https://www.safeside...
 Moving on to next url
Unable to complete save: local variable 'response' referenced before assignment 
 Moving onto next image...
Unable to complete save: cannot identify image file <_io.BytesIO object at 0x1107d4a40> 
 Moving onto next image...
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_8333787.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_4046580.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_5469195.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_5177415.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_5110206.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_5441748.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_5788978.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock

Traceback (most recent call last):
  File "/Users/ems/.pyenv/versions/3.8.5/lib/python3.8/site-packages/urllib3/connectionpool.py", line 446, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/Users/ems/.pyenv/versions/3.8.5/lib/python3.8/site-packages/urllib3/util/response.py", line 71, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'X-Frame-Options : SAMEORIGIN\r\nX-Content-Type-Options : nosniff\r\nX-Xss-Protection : 1; mode=block\r\nDate: Fri, 06 Nov 2020 13:42:10 GMT\r\nContent-Length: 87802\r\n\r\n'


Unable to download image from https://www.safeside...
 Moving on to next url
Unable to complete save: local variable 'response' referenced before assignment 
 Moving onto next image...
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_1194785.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_5842431.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_5832706.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_3484326.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_3667132.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_3607814.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_2471845.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_4028296.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/glock/glock_6739376.jpg
Saved image to

Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_5347789.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_9493353.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_8961499.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_3934354.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_9418261.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_9006879.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_9678027.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_9778965.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_5259745.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_3856180.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_9743286.jpg
Saved image to /Users

Traceback (most recent call last):
  File "/Users/ems/.pyenv/versions/3.8.5/lib/python3.8/site-packages/urllib3/connectionpool.py", line 446, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/Users/ems/.pyenv/versions/3.8.5/lib/python3.8/site-packages/urllib3/util/response.py", line 71, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'X-Frame-Options : SAMEORIGIN\r\nX-Content-Type-Options : nosniff\r\nX-Xss-Protection : 1; mode=block\r\nDate: Fri, 06 Nov 2020 13:50:28 GMT\r\nContent-Length: 282549\r\n\r\n'


Unable to download image from https://www.safeside...
 Moving on to next url
Unable to complete save: local variable 'response' referenced before assignment 
 Moving onto next image...
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_8107904.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_7263534.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_2086825.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_8159368.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_8377158.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_1690084.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_5896684.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_4398540.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_9086730.jpg
Saved image to

Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_4100337.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_6110259.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_1266571.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_5432976.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_1892169.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_5140552.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_7933791.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_2012093.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_2530207.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_2104547.jpg
Saved image to /Users/ems/Desktop/full_scraping_test/images/guns/AR-15/AR-15_4263237.jpg
Saved image to /Users

In [46]:
#Run this code block to execute full script
# wd = webdriver.Chrome(executable_path=DRIVER_PATH)
# urls_test = collect_urls("cats", 50, pause_time, wd)

ERROR! Session/line number was not unique in database. History logging moved to new session 66


In [ ]:
# def test(query, n_urls, pause, wd):
   
#     #Google Query Construction
#     search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

#     # load the page
#     wd.get(search_url.format(q=query))
# wd = webdriver.Chrome(executable_path=DRIVER_PATH)
# test('dog', 300, 5, wd)
print(count_total_thumbnails(wd))
# collect_thumbnails(wd)


In [9]:
# cwd = os.getcwd()  # Get the current working directory (cwd)
# files = os.listdir(cwd)  # Get all the files iddn that directory
# print("Files in %r: %s" % (cwd, files))